# ES → BBF Billing Account Migration

This notebook creates new BBF billing accounts for all ES accounts with active services.

## Process Overview
1. Query all `billing_invoice__c` with active services
2. For each, get the SF Account (`sfprod.account`) data
3. Create new OSS Account (`customers.accounts`) with `bbf_ban = true`
4. Create new SF Billing Invoice (`billing_invoice__c`) with:
   - `bbf_ban__c = true`
   - `Legacy_ES_Id__c` = lookup to original billing invoice SFID
5. Poll for SFID assignment
6. Output mapping to Excel

## Safety
- `DRY_RUN = True` by default
- Dry run outputs Excel showing what would be created
- Live run creates accounts and outputs mapping

In [32]:
# === SETUP ===
import psycopg2
from psycopg2.extras import RealDictCursor
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from datetime import datetime
from collections import defaultdict
import time
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Entity mapping (from original sync)
entity_map = {
    "Everstream Ohio": "EVO",
    "Everstream Michigan": "EVM",
    "OneCommunity": "EVO",
    "Medina County Fiber Network Services": "OCO",
    "Everstream Lynx": "EVL",
    "Michigan-LB": "EVB",
    "Michigan-RF": "EVR",
    "Everstream PA": "EVP",
}

# Connection credentials
heroku_conn_kwargs = {
    "dbname": "d88otjf7uhv9pr",
    "user": "ucn7cbk14sd6h",
    "password": "pf27d102f95e996e621e02523d035a1bff27590c8e6a13f5b180703a6631320c5",
    "host": "ec2-54-86-217-174.compute-1.amazonaws.com",
    "port": "5432",
    "cursor_factory": RealDictCursor,
    "connect_timeout": 10,
}

oss_conn_kwargs = {
    "dbname": "GLC",
    "user": "oss_server",
    "password": "3wU3uB28X?!r2?@ebrUg",
    "host": "pg01.comlink.net",
    "port": "5432",
    "cursor_factory": RealDictCursor,
    "connect_timeout": 10,
}

print("Connecting to Heroku...")
conn = psycopg2.connect(**heroku_conn_kwargs)
print("✅ Connected to Heroku")

print("Connecting to OSS...")
oconn = psycopg2.connect(**oss_conn_kwargs)
print("✅ Connected to OSS")

Connecting to Heroku...
✅ Connected to Heroku
Connecting to OSS...
✅ Connected to OSS


In [26]:
# === CONFIGURATION ===
DRY_RUN = True  # Set to False to actually create accounts
OUTPUT_DIR = "./data-migration"  # Change to your preferred output directory

# Output filename with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
run_type = "dry_run" if DRY_RUN else "live_run"
output_file = f"{OUTPUT_DIR}/es_bbf_account_clone_{run_type}_{timestamp}.xlsx"

print(f"DRY_RUN: {DRY_RUN}")
print(f"Output file: {output_file}")

DRY_RUN: True
Output file: ./data-migration/es_bbf_account_clone_dry_run_20251209_115728.xlsx


In [27]:
# === STEP 1: Get all billing_invoice__c with active services ===

active_billing_accounts_sql = """
SELECT DISTINCT 
    bi.account_number__c AS old_account_id,
    bi.sfid AS old_billing_invoice_sfid,
    bi.account__c AS customer_sfid,
    bi.name AS old_ban_name,
    bi.account_name__c,
    bi.billing_address_1__c,
    bi.billing_city__c,
    bi.billing_state__c,
    bi.billing_zip__c,
    bi.billing_e_mail__c,
    bi.late_fee_percentage__c,
    bi.disable_late_fees__c,
    bi.payment_terms__c,
    bi.invoice_delivery_preference__c,
    COUNT(o.sfid) AS active_order_count
FROM sfprod.billing_invoice__c bi
JOIN sfprod."order" o ON o.billing_invoice__c = bi.sfid
WHERE 
    o.status IN ('Activated', 'Disconnect in Progress', 'Suspended (Late Payment)')
    AND (o.service_end_date__c IS NULL OR o.service_end_date__c > CURRENT_DATE)
    AND bi.disabled__c = false
    AND bi.account_number__c IS NOT NULL
    AND (bi.bbf_ban__c IS NULL OR bi.bbf_ban__c = false)  -- Exclude already-migrated accounts
GROUP BY 
    bi.account_number__c, bi.sfid, bi.account__c, bi.name, bi.account_name__c,
    bi.billing_address_1__c, bi.billing_city__c, bi.billing_state__c, 
    bi.billing_zip__c, bi.billing_e_mail__c, bi.late_fee_percentage__c, 
    bi.disable_late_fees__c, bi.payment_terms__c, bi.invoice_delivery_preference__c
ORDER BY bi.account_number__c;
"""

with conn.cursor(cursor_factory=RealDictCursor) as cur:
    cur.execute(active_billing_accounts_sql)
    active_billing_accounts = cur.fetchall()

print(
    f"Found {len(active_billing_accounts)} ES billing accounts with active services to migrate."
)

Found 2522 ES billing accounts with active services to migrate.


In [28]:
# === STEP 2: Get SF Account data for each billing account ===

# Get unique customer SFIDs
customer_sfids = list(
    set([ba["customer_sfid"] for ba in active_billing_accounts if ba["customer_sfid"]])
)
customer_sfids_str = "','".join(customer_sfids)

sf_accounts_sql = f"""
SELECT 
    sfid,
    name,
    billingstreet,
    billingcity,
    billingstate,
    billingpostalcode,
    email_address__c,
    onecommunity_entity__c,
    business_sector__c,
    smartystreets__billing_address_status__c,
    send_consolidated_invoice__c
FROM sfprod.account
WHERE sfid IN ('{customer_sfids_str}')
"""

with conn.cursor(cursor_factory=RealDictCursor) as cur:
    cur.execute(sf_accounts_sql)
    sf_accounts = {row["sfid"]: row for row in cur.fetchall()}

print(f"Retrieved {len(sf_accounts)} SF Account records.")

Retrieved 2276 SF Account records.


In [29]:
# === STEP 3: Get OSS Customer records ===

oss_customers_sql = f"""
SELECT 
    customer_id,
    customer_nm,
    salesforce_id
FROM customers.customers
WHERE salesforce_id IN ('{customer_sfids_str}')
  AND disabled >= now()
"""

with oconn.cursor(cursor_factory=RealDictCursor) as ocur:
    ocur.execute(oss_customers_sql)
    oss_customers = {row["salesforce_id"]: row for row in ocur.fetchall()}

print(f"Retrieved {len(oss_customers)} OSS Customer records.")

# Check for missing customers
missing_customers = [sfid for sfid in customer_sfids if sfid not in oss_customers]
if missing_customers:
    print(
        f"⚠️ Warning: {len(missing_customers)} SF Accounts have no OSS Customer record"
    )

Retrieved 2275 OSS Customer records.
⚠️ Warning: 1 SF Accounts have no OSS Customer record


In [30]:
# === DEBUG: Show SF Accounts missing OSS Customer records ===

missing_customer_details = []

for sfid in missing_customers:
    sf_acct = sf_accounts.get(sfid)
    if sf_acct:
        # Find which billing accounts reference this SF Account
        related_bans = [
            ba for ba in active_billing_accounts if ba["customer_sfid"] == sfid
        ]

        missing_customer_details.append(
            {
                "sf_account_sfid": sfid,
                "sf_account_name": sf_acct.get("name"),
                "entity": sf_acct.get("onecommunity_entity__c"),
                "billing_city": sf_acct.get("billingcity"),
                "billing_state": sf_acct.get("billingstate"),
                "related_ban_count": len(related_bans),
                "related_ban_ids": ", ".join(
                    [str(ba["old_account_id"]) for ba in related_bans]
                ),
            }
        )

missing_df = pd.DataFrame(missing_customer_details)
print(f"SF Accounts with no OSS Customer record ({len(missing_df)}):\n")
display(missing_df)

SF Accounts with no OSS Customer record (1):



,sf_account_sfid,sf_account_name,entity,billing_city,billing_state,related_ban_count,related_ban_ids
0,0014P00002T46wSQAR,Paramount Schools of Excellence,Everstream Michigan,Indianapolis,IN,1,112788


In [33]:
# === DEBUG: Investigate missing OSS Customer links ===

# Get the account_ids for the affected billing accounts
affected_ban_ids = []
for sfid in missing_customers:
    related_bans = [ba for ba in active_billing_accounts if ba["customer_sfid"] == sfid]
    for ba in related_bans:
        affected_ban_ids.append(ba["old_account_id"])

print(f"Affected BAN IDs: {affected_ban_ids}\n")

# Check OSS: What customer_id do these accounts have?
affected_ids_str = ",".join([str(x) for x in affected_ban_ids])

oss_check_sql = f"""
SELECT 
    a.account_id,
    a.account_nm,
    a.customer_id,
    c.customer_nm,
    c.salesforce_id AS customer_salesforce_id,
    c.disabled AS customer_disabled
FROM customers.accounts a
LEFT JOIN customers.customers c ON a.customer_id = c.customer_id
WHERE a.account_id IN ({affected_ids_str})
ORDER BY a.account_id
"""

with oconn.cursor(cursor_factory=RealDictCursor) as ocur:
    ocur.execute(oss_check_sql)
    oss_results = ocur.fetchall()

print("OSS Account → Customer mapping:\n")
oss_df = pd.DataFrame(oss_results)
display(oss_df)

# Now check: What SF Account SFIDs are we looking for?
print("\n" + "=" * 50)
print("SF Account SFIDs we're looking for:\n")
for sfid in missing_customers:
    sf_acct = sf_accounts.get(sfid)
    print(f"  {sfid} → {sf_acct.get('name') if sf_acct else 'NOT FOUND'}")

Affected BAN IDs: ['112788']

OSS Account → Customer mapping:



,account_id,account_nm,customer_id,customer_nm,customer_salesforce_id,customer_disabled
0,112788,Paramount Schools of Excellence,112787,Paramount Schools of Excellence,001Rn00000KbCUwIAN,9999-12-31 23:59:59.999999+00:00



SF Account SFIDs we're looking for:

  0014P00002T46wSQAR → Paramount Schools of Excellence


In [22]:
# === DEBUG: Compare OSS Customer salesforce_id vs SF Account sfid ===

print("Comparing OSS Customer salesforce_id with expected SF Account sfid:\n")

comparison = []
for idx, row in oss_df.iterrows():
    account_id = row["account_id"]
    oss_customer_sfid = row["customer_salesforce_id"]

    # Find the expected SF Account sfid from the billing account
    ba = next(
        (
            ba
            for ba in active_billing_accounts
            if str(ba["old_account_id"]) == str(account_id)
        ),
        None,
    )
    expected_sfid = ba["customer_sfid"] if ba else None

    match_status = "MATCH" if oss_customer_sfid == expected_sfid else "MISMATCH"
    if oss_customer_sfid is None:
        match_status = "MISSING"

    comparison.append(
        {
            "account_id": account_id,
            "account_nm": row["account_nm"],
            "customer_id": row["customer_id"],
            "oss_salesforce_id": oss_customer_sfid,
            "expected_sf_sfid": expected_sfid,
            "status": match_status,
            "customer_disabled": row["customer_disabled"],
        }
    )

comp_df = pd.DataFrame(comparison)
display(comp_df)

# Summary
print("\n" + "=" * 50)
print("Summary:")
print(f"  MATCH: {len(comp_df[comp_df['status'] == 'MATCH'])}")
print(f"  MISMATCH: {len(comp_df[comp_df['status'] == 'MISMATCH'])}")
print(f"  MISSING: {len(comp_df[comp_df['status'] == 'MISSING'])}")

Comparing OSS Customer salesforce_id with expected SF Account sfid:



,account_id,account_nm,customer_id,oss_salesforce_id,expected_sf_sfid,status,customer_disabled
0,162,Ligonier Telephone,162,001U000001jMJTzIAO,0010B00001oAgq4QAC,MISMATCH,9999-12-31 23:59:59.999999+00:00
1,101416,Lansing Fiber Communications - Zayo,101417,None,001Qp00000pDqugIAC,MISSING,9999-12-31 23:59:59.999999+00:00
2,111413,Law Office of Crystal L. Johnson,111091,0014P00002cdTOnQAM,0013g00000JZ3jGAAT,MISMATCH,9999-12-31 23:59:59.999999+00:00
3,115937,W-LOK Corporation,115938,None,0016g00002hpxKsAAI,MISSING,9999-12-31 23:59:59.999999+00:00
4,116589,Paramount Schools of Excellence,112787,0014P00002T46wSQAR,001Rn00000KbCUwIAN,MISMATCH,9999-12-31 23:59:59.999999+00:00
5,116875,connectSCP/South Central Power Company,116876,None,001Rn00000EVykbIAD,MISSING,9999-12-31 23:59:59.999999+00:00



Summary:
  MATCH: 0
  MISMATCH: 3
  MISSING: 3


In [ ]:
# === FIX: Update OSS Customer salesforce_id ===

DRY_RUN_FIX = True  # Set to False to actually update

fixes = [
    {
        "customer_id": 162,
        "new_salesforce_id": "0010B00001oAgq4QAC",
        "name": "Ligonier Telephone",
    },
    {
        "customer_id": 101417,
        "new_salesforce_id": "001Qp00000pDqugIAC",
        "name": "Lansing Fiber Communications - Zayo",
    },
    {
        "customer_id": 111091,
        "new_salesforce_id": "0013g00000JZ3jGAAT",
        "name": "Law Office of Crystal L. Johnson",
    },
    {
        "customer_id": 115938,
        "new_salesforce_id": "0016g00002hpxKsAAI",
        "name": "W-LOK Corporation",
    },
    {
        "customer_id": 112787,
        "new_salesforce_id": "001Rn00000KbCUwIAN",
        "name": "Paramount Schools of Excellence",
    },
    {
        "customer_id": 116876,
        "new_salesforce_id": "001Rn00000EVykbIAD",
        "name": "connectSCP/South Central Power Company",
    },
]

if DRY_RUN_FIX:
    print("DRY RUN - Would update the following:\n")
    for fix in fixes:
        print(f"  customer_id {fix['customer_id']} ({fix['name']})")
        print(f"    → salesforce_id = '{fix['new_salesforce_id']}'\n")
else:
    with oconn.cursor() as ocur:
        for fix in fixes:
            ocur.execute(
                """
                UPDATE customers.customers 
                SET salesforce_id = %s 
                WHERE customer_id = %s
            """,
                (fix["new_salesforce_id"], fix["customer_id"]),
            )
            print(f"✅ Updated customer_id {fix['customer_id']} ({fix['name']})")

        oconn.commit()
        print("\n✅ All updates committed.")

✅ Updated customer_id 162 (Ligonier Telephone)
✅ Updated customer_id 101417 (Lansing Fiber Communications - Zayo)
✅ Updated customer_id 111091 (Law Office of Crystal L. Johnson)
✅ Updated customer_id 115938 (W-LOK Corporation)
✅ Updated customer_id 112787 (Paramount Schools of Excellence)
✅ Updated customer_id 116876 (connectSCP/South Central Power Company)

✅ All updates committed.


In [ ]:
# === HELPER FUNCTIONS (from original sync) ===


def create_oss_account(sf_account_info, customer_id, ocur):
    """
    Creates a new OSS billing account using SF Account data.
    Mirrors _create_oss_account from sync_orders_from_sf.ipynb
    """
    # Validate address
    address_status = (
        sf_account_info.get("smartystreets__billing_address_status__c") or ""
    )
    if "Confirm" not in address_status:
        return None, "Bad/Missing/Unconfirmed Address"

    # Parse zip code
    billing_zip = sf_account_info.get("billingpostalcode") or ""
    if "-" in billing_zip:
        zip_cd, zip4 = billing_zip.split("-", 1)
    else:
        zip_cd = billing_zip
        zip4 = None

    # Get entity code
    entity = sf_account_info.get("onecommunity_entity__c")
    if entity not in entity_map:
        return None, f"Unknown entity: {entity}"
    company_cd = entity_map[entity]

    # Determine customer type
    business_sector = sf_account_info.get("business_sector__c") or ""
    customer_type_cd = "W" if business_sector == "Wholesale" else "R"

    # Insert new account
    ocur.execute(
        """
        INSERT INTO customers.accounts (
            account_nm,
            customer_id,
            company_cd,
            address1,
            city,
            state_cd,
            zip,
            zip4,
            billing_email,
            created_by_id,
            late_fee_percentage,
            customer_type_cd,
            bbf_ban
        ) VALUES (
            %s, %s, %s, %s, %s, %s, %s, %s, %s, 0, %s, %s, true
        ) RETURNING *
    """,
        (
            sf_account_info["name"],
            customer_id,
            company_cd,
            sf_account_info.get("billingstreet"),
            sf_account_info.get("billingcity"),
            sf_account_info.get("billingstate"),
            zip_cd,
            zip4,
            sf_account_info.get("email_address__c"),
            0.015,  # Default late fee percentage
            customer_type_cd,
        ),
    )

    return ocur.fetchone(), None


def create_sf_billing_account(
    oss_account_info, customer_sfid, legacy_billing_sfid, cur
):
    """
    Creates a new SF Billing Invoice record in Heroku.
    Mirrors _create_sf_account from sync_orders_from_sf.ipynb
    Adds bbf_ban__c = true and Legacy_ES_Id__c = original billing invoice SFID
    """
    # Build account number (matches original format)
    account_number = f"A{oss_account_info['account_id']}"

    # Determine invoice delivery preference
    paperless = oss_account_info.get("paperless", False)
    email_non_paperless = oss_account_info.get("email_non_paperless", False)

    if paperless:
        delivery_pref = "E-mail"
    elif not paperless and email_non_paperless:
        delivery_pref = "E-mail & Paper"
    else:
        delivery_pref = "Paper"

    cur.execute(
        """
        INSERT INTO sfprod.billing_invoice__c (
            billing_city__c,
            billing_address_1__c,
            account__c,
            account_number__c,
            name,
            late_fee_percentage__c,
            billing_zip__c,
            billing_state__c,
            account_name__c,
            billing_address_2__c,
            disable_late_fees__c,
            suppress_invoice_generation__c,
            payment_terms__c,
            suppress_past_due_notifications__c,
            invoice_delivery_preference__c,
            billing_e_mail__c,
            bbf_ban__c,
            legacy_es_id__c
        ) VALUES (
            %(city)s,
            %(address1)s,
            %(customer_sfid)s,
            %(account_id)s,
            %(account_number)s,
            %(late_fee_percentage)s,
            %(zip)s,
            %(state_cd)s,
            %(account_nm)s,
            %(address2)s,
            %(disable_late_fees)s,
            false,
            %(due_date_frequency_cd)s,
            %(no_past_due_notice)s,
            %(delivery_pref)s,
            %(billing_email)s,
            true,
            %(legacy_billing_sfid)s
        )
        ON CONFLICT (account_number__c) DO NOTHING
    """,
        {
            "city": oss_account_info.get("city"),
            "address1": oss_account_info.get("address1"),
            "customer_sfid": customer_sfid,
            "account_id": str(oss_account_info["account_id"]),
            "account_number": account_number,
            "late_fee_percentage": oss_account_info.get("late_fee_percentage", 0.015),
            "zip": oss_account_info.get("zip"),
            "state_cd": oss_account_info.get("state_cd"),
            "account_nm": oss_account_info.get("account_nm"),
            "address2": oss_account_info.get("address2"),
            "disable_late_fees": oss_account_info.get("disable_late_fees", False),
            "due_date_frequency_cd": oss_account_info.get(
                "due_date_frequency_cd", "NET30"
            ),
            "no_past_due_notice": oss_account_info.get("no_past_due_notice", False),
            "delivery_pref": delivery_pref,
            "billing_email": oss_account_info.get("billing_email"),
            "legacy_billing_sfid": legacy_billing_sfid,
        },
    )


def poll_for_sfid(account_id, customer_sfid, cur, max_retries=5, sleep_seconds=5):
    """
    Polls Heroku waiting for Salesforce to assign an SFID to the new billing invoice.
    Mirrors the retry logic from _create_sf_account in sync_orders_from_sf.ipynb
    """
    new_billing_acct = None
    retry_count = 0

    while retry_count <= max_retries and (
        new_billing_acct is None or new_billing_acct.get("sfid") is None
    ):
        retry_count += 1
        cur.execute(
            """
            SELECT sfid, account__c, account_number__c
            FROM sfprod.billing_invoice__c
            WHERE account__c = %s
              AND account_number__c = %s
        """,
            (customer_sfid, str(account_id)),
        )

        billing_acct = cur.fetchone()
        if billing_acct and billing_acct.get("sfid"):
            new_billing_acct = billing_acct
        else:
            time.sleep(sleep_seconds)

    return new_billing_acct

In [ ]:
# === STEP 4: Process each billing account ===

results = []
account_mapping = {}  # old_account_id -> new_account_id

total = len(active_billing_accounts)
for idx, ba in enumerate(active_billing_accounts, 1):
    old_account_id = ba["old_account_id"]
    customer_sfid = ba["customer_sfid"]
    old_billing_sfid = ba["old_billing_invoice_sfid"]

    result = {
        "old_account_id": old_account_id,
        "old_billing_invoice_sfid": old_billing_sfid,
        "account_name": ba["account_name__c"],
        "old_ban_name": ba["old_ban_name"],
        "billing_city": ba["billing_city__c"],
        "billing_state": ba["billing_state__c"],
        "billing_email": ba["billing_e_mail__c"],
        "active_order_count": ba["active_order_count"],
        "customer_sfid": customer_sfid,
        "new_account_id": None,
        "new_billing_invoice_sfid": None,
        "new_ban_name": None,
        "status": None,
        "error": None,
    }

    # Get SF Account
    sf_account = sf_accounts.get(customer_sfid)
    if not sf_account:
        result["status"] = "SKIPPED"
        result["error"] = "SF Account not found"
        results.append(result)
        continue

    # Get OSS Customer
    oss_customer = oss_customers.get(customer_sfid)
    if not oss_customer:
        result["status"] = "SKIPPED"
        result["error"] = "OSS Customer not found"
        results.append(result)
        continue

    customer_id = oss_customer["customer_id"]

    # === DRY RUN: Just record what would happen ===
    if DRY_RUN:
        # Validate what we can
        address_status = (
            sf_account.get("smartystreets__billing_address_status__c") or ""
        )
        if "Confirm" not in address_status:
            result["status"] = "WOULD_SKIP"
            result["error"] = "Bad/Missing/Unconfirmed Address"
        else:
            entity = sf_account.get("onecommunity_entity__c")
            if entity not in entity_map:
                result["status"] = "WOULD_SKIP"
                result["error"] = f"Unknown entity: {entity}"
            else:
                result["new_account_id"] = "(pending)"
                result["new_billing_invoice_sfid"] = "(pending)"
                result["new_ban_name"] = "(pending)"
                result["status"] = "WOULD_CREATE"

        results.append(result)

        if idx % 100 == 0:
            print(f"Processed {idx}/{total}...")
        continue

    # === LIVE RUN: Actually create accounts ===
    try:
        print(
            f"[{idx}/{total}] Creating account for {ba['account_name__c']} (old: {old_account_id})"
        )

        # Step 1: Create OSS Account
        with oconn.cursor(cursor_factory=RealDictCursor) as ocur:
            new_oss_account, error = create_oss_account(sf_account, customer_id, ocur)

            if error:
                result["status"] = "SKIPPED"
                result["error"] = error
                results.append(result)
                oconn.rollback()
                continue

            new_account_id = new_oss_account["account_id"]
            result["new_account_id"] = new_account_id

            # Commit OSS transaction so Heroku Connect can see it
            oconn.commit()
            print(f"  ✅ Created OSS account: {new_account_id}")

        # Step 2: Create SF Billing Invoice in Heroku
        with conn.cursor(cursor_factory=RealDictCursor) as cur:
            create_sf_billing_account(
                new_oss_account, customer_sfid, old_billing_sfid, cur
            )
            conn.commit()
            print(f"  ✅ Created SF Billing Invoice record")

        # Step 3: Poll for SFID
        with conn.cursor(cursor_factory=RealDictCursor) as cur:
            new_billing_acct = poll_for_sfid(new_account_id, customer_sfid, cur)

            if new_billing_acct and new_billing_acct.get("sfid"):
                result["new_billing_invoice_sfid"] = new_billing_acct["sfid"]
                result["new_ban_name"] = f"A{new_account_id}"
                result["status"] = "CREATED"
                account_mapping[old_account_id] = new_account_id
                print(f"  ✅ Got SFID: {new_billing_acct['sfid']}")
            else:
                result["status"] = "CREATED_NO_SFID"
                result["error"] = "Timed out waiting for SFID"
                account_mapping[old_account_id] = new_account_id
                print(f"  ⚠️ Created but no SFID yet")

    except Exception as e:
        result["status"] = "ERROR"
        result["error"] = str(e)
        oconn.rollback()
        conn.rollback()
        print(f"  ❌ Error: {e}")

    results.append(result)

# Summary
print("\n" + "=" * 50)
print("SUMMARY")
print("=" * 50)
status_counts = defaultdict(int)
for r in results:
    status_counts[r["status"]] += 1

for status, count in sorted(status_counts.items()):
    print(f"{status}: {count}")
print(f"\nTotal processed: {len(results)}")

In [ ]:
# === STEP 5: Create output Excel file ===

wb = Workbook()

# --- Sheet 1: Account Clone Results ---
ws1 = wb.active
ws1.title = "Clone Results"

# Styling
header_font = Font(bold=True, color="FFFFFF")
header_fill = PatternFill("solid", fgColor="4472C4")
header_alignment = Alignment(horizontal="center", vertical="center", wrap_text=True)
thin_border = Border(
    left=Side(style="thin"),
    right=Side(style="thin"),
    top=Side(style="thin"),
    bottom=Side(style="thin"),
)

# Headers
headers = [
    "Old Account ID",
    "New Account ID",
    "Account Name",
    "Old BAN Name",
    "New BAN Name",
    "Old Billing Invoice SFID",
    "New Billing Invoice SFID",
    "City",
    "State",
    "Email",
    "Active Orders",
    "Customer SFID",
    "Status",
    "Error",
]
ws1.append(headers)

for col, header in enumerate(headers, 1):
    cell = ws1.cell(row=1, column=col)
    cell.font = header_font
    cell.fill = header_fill
    cell.alignment = header_alignment
    cell.border = thin_border

# Status colors
status_colors = {
    "WOULD_CREATE": "FFF2CC",  # Light yellow (dry run - would create)
    "WOULD_SKIP": "FCE4D6",  # Light orange (dry run - would skip)
    "CREATED": "C6EFCE",  # Light green (success)
    "CREATED_NO_SFID": "FFEB9C",  # Yellow (partial success)
    "SKIPPED": "FFCCCC",  # Light red (skipped)
    "ERROR": "FF6666",  # Red (error)
}

# Data rows
for row_idx, r in enumerate(results, 2):
    row_data = [
        r["old_account_id"],
        r["new_account_id"],
        r["account_name"],
        r["old_ban_name"],
        r["new_ban_name"],
        r["old_billing_invoice_sfid"],
        r["new_billing_invoice_sfid"],
        r["billing_city"],
        r["billing_state"],
        r["billing_email"],
        r["active_order_count"],
        r["customer_sfid"],
        r["status"],
        r["error"],
    ]
    ws1.append(row_data)

    fill_color = status_colors.get(r["status"], "FFFFFF")
    for col in range(1, len(headers) + 1):
        cell = ws1.cell(row=row_idx, column=col)
        cell.fill = PatternFill("solid", fgColor=fill_color)
        cell.border = thin_border

# Auto-fit columns
for col in ws1.columns:
    max_length = 0
    column = col[0].column_letter
    for cell in col:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(str(cell.value))
        except:
            pass
    ws1.column_dimensions[column].width = min(max_length + 2, 40)

# Freeze header row
ws1.freeze_panes = "A2"

# --- Sheet 2: Summary ---
ws2 = wb.create_sheet("Summary")
ws2.append(["ES → BBF Account Migration Summary"])
ws2["A1"].font = Font(bold=True, size=14)
ws2.append([])
ws2.append(["Run Type:", "DRY RUN" if DRY_RUN else "LIVE RUN"])
ws2.append(["Timestamp:", datetime.now().strftime("%Y-%m-%d %H:%M:%S")])
ws2.append([])
ws2.append(["Status", "Count"])
ws2["A6"].font = Font(bold=True)
ws2["B6"].font = Font(bold=True)

for status, count in sorted(status_counts.items()):
    ws2.append([status, count])

ws2.append([])
ws2.append(["Total Accounts Processed:", len(results)])
ws2.append([])

# Color legend
ws2.append(["Status Color Legend:"])
ws2.append(["WOULD_CREATE", "Account would be created (dry run)"])
ws2.append(["WOULD_SKIP", "Account would be skipped (dry run)"])
ws2.append(["CREATED", "Account successfully created"])
ws2.append(["CREATED_NO_SFID", "Account created, waiting for SF sync"])
ws2.append(["SKIPPED", "Account skipped due to validation"])
ws2.append(["ERROR", "Error during creation"])

# --- Sheet 3: Account Mapping (for future order migration) ---
ws3 = wb.create_sheet("Account Mapping")
mapping_headers = [
    "Old Account ID",
    "New Account ID",
    "Old Billing Invoice SFID",
    "New Billing Invoice SFID",
    "Customer SFID",
    "Account Name",
]
ws3.append(mapping_headers)

for col, header in enumerate(mapping_headers, 1):
    cell = ws3.cell(row=1, column=col)
    cell.font = header_font
    cell.fill = header_fill
    cell.alignment = header_alignment

for r in results:
    if r["status"] in ("WOULD_CREATE", "CREATED", "CREATED_NO_SFID"):
        ws3.append(
            [
                r["old_account_id"],
                r["new_account_id"],
                r["old_billing_invoice_sfid"],
                r["new_billing_invoice_sfid"],
                r["customer_sfid"],
                r["account_name"],
            ]
        )

# Freeze header
ws3.freeze_panes = "A2"

# Save
wb.save(output_file)
print(f"\n✅ Output saved to: {output_file}")

In [ ]:
# === STEP 6: Cleanup ===

if DRY_RUN:
    print("\n🔄 DRY RUN complete. No changes were made.")
    print(f"Review the output file: {output_file}")
    print("\nTo run for real, set DRY_RUN = False and re-run the notebook.")
else:
    print(f"\n✅ LIVE RUN complete.")
    print(
        f"Created {status_counts.get('CREATED', 0) + status_counts.get('CREATED_NO_SFID', 0)} new accounts."
    )
    print(f"Output saved to: {output_file}")

In [ ]:
# === Close connections ===
conn.close()
oconn.close()
print("🔌 Database connections closed.")

---
## Future: Move Orders to New BBF Accounts

When the merger closes, use the Account Mapping sheet from this output to move active orders from old ES accounts to new BBF accounts.

The order migration will:
1. Load the account mapping from the Excel file
2. For each old account, find active orders
3. Update `om.orders.account_id` to the new BBF account
4. Optionally filter by PA/Non-PA market

In [ ]:
# === PLACEHOLDER: Order Migration (to be implemented when merger closes) ===

PA_MARKETS = ["Pittsburgh", "Harrisburg", "Philadelphia", "Scranton", "Uniti-PA"]

# def move_orders_to_bbf_accounts(mapping_file, filter_pa=False, dry_run=True):
#     """
#     Moves active orders from ES accounts to BBF accounts.
#
#     Args:
#         mapping_file: Path to Excel file with Account Mapping sheet
#         filter_pa: If True, only move Non-PA orders
#         dry_run: If True, don't actually move orders
#     """
#     pass

print("⏳ Order migration not yet implemented. Will be added when merger closes.")

In [6]:
import sys

print(sys.executable)
!pip install openpyxl

C:\Users\vjero\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe
Defaulting to user installation because normal site-packages is not writeable
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\vjero\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
